In [1]:
!pip install "dask[dataframe]"
!pip install tqdm

    100% |################################| 583kB 2.2MB/s ta 0:00:011
    100% |################################| 51kB 10.4MB/s ta 0:00:01
  Running setup.py bdist_wheel for toolz ... done
  Stored in directory: /root/.cache/pip/wheels/57/51/8a/433a9c0a2c65fc1b2a795ae036b932f3339a02e9ae88367659
  Running setup.py bdist_wheel for locket ... done
  Stored in directory: /root/.cache/pip/wheels/7a/de/da/8039d7d9547aabfb95f2deca20948c8eb4c71c3b85405547c2
Successfully built toolz locket
You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |################################| 61kB 5.6MB/s ta 0:00:011
You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import dask.array as da
import h5py as h5

from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import fbeta_score, accuracy_score
from sklearn.externals import joblib

from tqdm import tqdm

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [109]:
# Start client
# client = Client('10.70.1.160:8786')
# client = Client(processes=False)

In [58]:
# Parameters
DATA_PATH = r'data/eightieth.h5' # Path to the learning dataset
CHUNK_SIZE = 15000 # Number of images to process in one batch (must fit comfortably in memory)
TRAIN_SIZE = 7/8 # Size of the train dataset (compared to the total dataset)


# data_h5 = h5.File('data/eightieth.h5', mode='r')
# landcover = da.from_array(data_h5['TOP_LANDCOVER'], chunks=(1000,1))
# images = da.from_array(data_h5['S2'], chunks=(1000,16,16,4))

In [59]:
# Setup generator yielding X, y chunks (default size 1000)

classes = da.unique(da.from_array(h5.File(DATA_PATH)['TOP_LANDCOVER'], chunks=(CHUNK_SIZE,1))).compute()

def generator(h5_path, chunk_size, train_size):
    f = h5.File(h5_path, 'r')
    X = f['S2']
    y = f['TOP_LANDCOVER']
    
    # Compute number of chunks needed
    no_chunks = len(X) // chunk_size
    if len(X) % chunk_size > 0:
        # len(X) is not an exact multiple of chunk_size
        no_chunks += 1
    
    print(len(X), '-', no_chunks)
    for c in range(no_chunks):
        X_to_yield = X[c*chunk_size:(c+1)*chunk_size,:,:,:]
        y_to_yield = y[c*chunk_size:(c+1)*chunk_size,:]
        
        # Wrangle data
        X_to_yield = np.array(X_to_yield).reshape((-1, 16*16*4)) # Flatten each element in the array
        y_to_yield = np.array(y_to_yield).astype(int).reshape((-1,))
        y_to_yield = label_binarize(y_to_yield, classes=classes)
        
        
        yield train_test_split(X_to_yield, y_to_yield, train_size=train_size, test_size=1-train_size)
    
# Test
# for X, y in generator('data/eightieth.h5', 1000):
#     pass

In [60]:
forests_list = [] # List of forests to aggregate

for X_train, X_test, y_train, y_test in tqdm(generator(DATA_PATH, CHUNK_SIZE, TRAIN_SIZE)):
    cf = RandomForestClassifier(n_estimators=20, n_jobs=-1) # parameters to define
    cf.fit(X=X_train, y=y_train)
    
#     Reshape y_test to make it understandable (1D series)
    score = cf.score(X_test, y_test)
    
    forests_list.append((cf, score))
    
forests_list = np.array(forests_list)
print('Mean score:', forests_list[:,1].mean())

0it [00:00, ?it/s]

234000 - 16


16it [00:51,  3.23s/it]

Mean score: 0.4153444444444444


In [61]:
# Custom VotingClassifier
# Can ensemble pre-fitted models (as opposed to sklearn's implementation which needs retraining)

class VotingClassifier:
    # Two voting modes :
        # - Hard: the majority wins, if there's a tie, a contender is selected randomly
        # - Soft: use the provided weights and tally the weighted votes to determine winners. Weights must have shape (len(estimators))
    
    def __init__(self, estimators, classes=None, weights=None):
        # estimators: iterable of estimators implementing the predict() API
        # weights: iterable of (numeric) weights
        # classes: list of matching classes
        
        self.estimators = estimators
        self.weights = np.array(weights)
        self.n_classes = len(self.estimators[0].n_classes_)
        
    
    def _special_argmax(self, pred):
        # Compute the usual argmax along the lines
        # For each null predictions ([0]*no_features), replace the argmax with -1 to filter it easily
        # Having an argmax of 0 then actually means that the predicted label is the #0

        # pred shape=(no_classifiers, no_features)
        out = pred.argmax(axis=1)
        out[np.all(pred==-1.0, axis=1)] = self.n_classes + 1

        return out

    def predict(self, X):
        # X is of shape (no_samples, no_features)
        # Return array of shape (no_samples,)

        predictions = np.empty((len(self.estimators), X.shape[0], self.n_classes))
        
        for i, clf in enumerate(self.estimators):
            predictions[i, :, :] = clf.predict(X)
            
        # predictions is of shape (no_classifiers, no_samples, no_features)
        
        out = np.zeros((X.shape[0],)) # out matrix
        
        for k in range(predictions.shape[1]):
            arr = predictions[:, k, :]
            
            # Replace all the 0 to -1 (in order to filter out the null prediction later on)
            np.place(arr, arr==0.0, -1)
            
            arr_label = self._special_argmax(arr)
            
            idx_label = ~(arr_label == self.n_classes + 1)
            
            arr_label = arr_label[idx_label]

            if len(arr_label) == 0:
                # If an image cannot be predicted at all by any estimator, we simply affect it a random class unfortunately
                choice = np.random.choice(range(self.n_classes))
                print('Unpredictable image at index {}, affected class -> {}'.format(k, choice))
                out[k] = choice
                continue
            
            out[k] = np.argmax(np.bincount(arr_label, 
                                           weights=weights[idx_label]))
            
        return out    
    
    def score(self, X, y):
        # Convert y to its category label
        y = y.argmax(axis=1)
        y_pred = self.predict(X)
        
        accuracy = accuracy_score(y, y_pred)
        fbeta = fbeta_score(y, y_pred, 2, average='macro')
        
        print('Accuracy :', accuracy)
        print('F2 score :', fbeta)

In [62]:
weights = forests_list[:,1].astype(float)
vc = VotingClassifier(forests_list[:,0], weights=weights)
# predictions = vc.predict(X_test)
vc.score(X_test, y_test)

Unpredictable image at index 11, affected class -> 0
Unpredictable image at index 25, affected class -> 9
Unpredictable image at index 36, affected class -> 8
Unpredictable image at index 56, affected class -> 2
Unpredictable image at index 68, affected class -> 9
Unpredictable image at index 71, affected class -> 11
Unpredictable image at index 74, affected class -> 9
Unpredictable image at index 78, affected class -> 7
Unpredictable image at index 97, affected class -> 3
Unpredictable image at index 101, affected class -> 1
Unpredictable image at index 173, affected class -> 11
Unpredictable image at index 178, affected class -> 1
Unpredictable image at index 198, affected class -> 3
Unpredictable image at index 203, affected class -> 3
Unpredictable image at index 212, affected class -> 2
Unpredictable image at index 213, affected class -> 10
Unpredictable image at index 232, affected class -> 8
Unpredictable image at index 237, affected class -> 7
Unpredictable image at index 239, 

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [64]:
# Write the model to the disk
joblib.dump(vc, 'vc_20trees_eigtieth.pkl')

['vc_20trees_eigtieth.pkl']